<a href="https://colab.research.google.com/github/archyrei/latihan-python/blob/main/make_face_signatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Membuat dataset *face signatures* FER**

---
menggunakan MTCNN dan FaceNet, untuk mendapatkan *face pattern*
dan dijadikan file csv


In [1]:
#@title Install MTCNN
!pip install git+https://github.com/archyrei/mtcnn.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/archyrei/mtcnn.git to /tmp/pip-req-build-gvyj83mb
  Running command git clone -q https://github.com/archyrei/mtcnn.git /tmp/pip-req-build-gvyj83mb
  Created wheel for mtcnn: filename=mtcnn-0.1.1-py3-none-any.whl size=2260437 sha256=29a6bfb5a77cf05636abfa18917d82edb5f9a8901de9fef2333b962fdef53d83
  Stored in directory: /tmp/pip-ephem-wheel-cache-kv74cnxy/wheels/24/fa/a4/644d1511f42c3be469c204ff9d7db04d55fcac7ccb25649009
Successfully built mtcnn


In [2]:
import os
import csv
from os import listdir
from PIL import Image as Img
from numpy import asarray
from numpy import expand_dims
from keras.models import load_model
import numpy as np
import mtcnn
from mtcnn.network.factory import NetworkFactory
import tensorflow as tf
import matplotlib.pyplot as plt

import pickle


In [6]:
#load keras model
iniFacenet = load_model('/content/drive/MyDrive/Colab Notebooks/model/facenet_keras.h5')

def extract_face(path):
  #buka gambar dan convert ke RGB
  gmbr = Img.open(path)
  gmbr = gmbr.convert('RGB')
  arrGmbr = asarray(gmbr)
  detector = mtcnn.MTCNN()
  #face detect
  face_locations = detector.detect_faces(arrGmbr)
  x1, y1, lebar, tinggi = face_locations[0]['box']
  x1, y1 = abs(x1), abs(y1)
  x2, y2 = x1 + lebar, y1 + tinggi
  #extract face
  iniFace = arrGmbr[y1:y2, x1:x2]
  #resize face
  imagi = Img.fromarray(iniFace)
  imagi = imagi.resize((160, 160))
  arrFace = asarray(imagi)
  #penyesuaian dan ...
  arrFace = arrFace.astype('float32')
  mean, std = arrFace.mean(), arrFace.std()
  arrFace = (arrFace - mean) / std
  arrFace = expand_dims(arrFace, axis=0)
  signature = iniFacenet.predict(arrFace)
  return signature[0].tolist()


# draw an image with detected objects
def draw_facebox(filename, result_list):
  data = plt.imread(filename)
  plt.imshow(data)
  ax = plt.gca()
  for result in result_list:
    x, y, width, height = result['box']
    rect = plt.Rectangle((x, y), width, height,fill=False, color='green',lw=1)
    ax.add_patch(rect)
    for key, value in result['keypoints'].items():
      dot = plt.Circle(value, radius=2, color='red')
      ax.add_patch(dot)
  plt.show()

# draw each face separately
def draw_faces(filename, result_list):
  # load the image
  data = plt.imread(filename)
  # plot each face as a subplot
  for i in range(len(result_list)):
    # get coordinates
    x1, y1, width, height = result_list[i]['box']
    x2, y2 = x1 + width, y1 + height
    # define subplot
    plt.subplot(1, len(result_list), i+1)
    plt.axis('off')
    # plot face
    plt.imshow(data[y1:y2, x1:x2])
  # show the plot
  plt.show()

In [11]:
#load image from drive

folder_name = "/content/drive/MyDrive/Gambar/fatigue/"
idx = 0

#print("Shape of image/array:", arrGmbr.shape)

#imgplot = plt.imshow(arrGmbr)
#plt.show()

In [18]:
csv_filename = '/content/drive/MyDrive/Colab Notebooks/signatures.csv'

with open(csv_filename, mode='w') as csv_file:
  #
  fieldnames = ['name','signature','label']
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
  writer.writeheader()

  for filename in listdir(folder_name):
    
    #key
    idx+=1
    path = folder_name + filename
    face_extract = extract_face(path)

    #make dataset csv
    writer.writerow({'name':'face{index}'.format(index=idx),'signature': face_extract,'label':'01'})

    #jdijson.update({"face{index}".format(index=idx) : signature[0].tolist()})
    

In [ ]:
# kosong

In [ ]:
# kosong

In [ ]:
# Writing to sample.json
#with open("/content/drive/MyDrive/code/data.json", "w") as outfile:
  #json.dump(jdijson, outfile, indent = 3)

In [ ]:
# kosong